# Direction selectivity analysis

In [1]:
import pynwb
import numpy as np
from datavyz import graph_env_manuscript as ge

In [27]:
def single_cell_response(filename, nROI=0):
    fig, AX = ge.figure(axes_extents=[[[3,1],[1,1]]], wspace=1, figsize=(1.5,3))